In [44]:
import os
import json
import psycopg2
from tqdm import tqdm
import time
import pandas as pd

In [ ]:
                        port=pg_credential.port,
                        user=pg_credential.username,
                        password=pg_credential.password,
                        database=pg_credential.path[1:]) # To remove slash

cursor = conn.cursor()
cursor.execute("INSERT INTO a_table (c1, c2, c3) VALUES(%s, %s, %s)", (v1, v2, v3))
cursor.close()
conn.close()

In [39]:
cursor.close()
conn.close()

# SQL APPROACH

In [81]:
from psycopg2.extras import Json

In [ ]:
def insert_sql_func(directory):
    reconnect_count = 0 
    conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
    cursor = conn.cursor()
    files_list = os.listdir(directory)
    for unigr_index in tqdm(range(912, len(files_list))):
        unigr = files_list[unigr_index]
        if unigr.endswith("json"):
            sim_words_list = []
            with open(os.path.join(directory,unigr)) as f:
                main_words = json.load(f)
                word = {}
                for main_word in main_words:
                    main_word_id = main_word['word_id']
                    main_ref = main_word['ref_id']
                    main_set = main_word['setting_id']
                    orig_word = main_word['word']
                    word['word_id'] = main_word['word_id']
                    word['ref_id'] = main_word['ref_id']
                    word['setting_id'] = main_word['setting_id']
                    word['ngramm'] = main_word['word']
                    word['simlar_words'] = []
                    for similar_word in main_word['simlar_words']:
                        sim_word = {}
                        sim_word['word_id'] = 0
                        sim_word['ref_id'] = 0
                        sim_word['setting_id'] = 0
                        sim_word['ngramm'] = similar_word
                        word['simlar_words'].append(sim_word)
                        #sim_words_list.append((main_word_id, main_ref, main_set, 0, 0,0,sim_word,orig_word))
                        #print(main_word_id, main_ref, main_set, sim_word)
                        #cursor.execute("INSERT INTO linguist.mix_locales (word_id, ref_id, setting_id, mix_word_id, mix_ref_id, mix_setting_id, mix_word,orig_word) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", (main_word_id, main_ref, main_set, 0, 0,0,sim_word,orig_word))
                        #conn.commit() 
                        #break
                        
                    #print(word)
                    sim_words_list.append(word)
                    #break
            #print(sim_words_list[0])
            #print(unigr,"all words collected, going to insert")
            #cursor.executemany("INSERT INTO linguist.mix_locales (word_id, ref_id, setting_id, mix_word_id, mix_ref_id, mix_setting_id, mix_word,orig_word) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", sim_words_list)
            #conn.commit()
            #print("inseted completed")
            #print(unigr,"all words collected, going to insert") 
            try:
                cursor.execute("SELECT linguist.add_locales_ngrams( %s ::jsonb)",(Json(sim_words_list),))
                conn.commit() 
            except:
                print(unigr_index, "crashed")
                cursor.close()
                conn.close()
                reconnect_count += 1
                if reconnect_count < 10:
                    time.sleep(5)
                    conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
                    cursor = conn.cursor()
                else:
                    print("reconnection limit")
            #print("insert completed")
        
    cursor.close()
    conn.close()
    return sim_words_list
        
words = insert_sql_func("./save_unigr_rus")















  0%|          | 0/1093 [00:00<?, ?it/s]











  0%|          | 1/1093 [00:03<56:37,  3.11s/it]











  0%|          | 2/1093 [00:04<46:02,  2.53s/it]











  0%|          | 3/1093 [00:05<38:43,  2.13s/it]











  0%|          | 4/1093 [00:06<32:55,  1.81s/it]











  0%|          | 5/1093 [00:07<28:45,  1.59s/it]











  1%|          | 6/1093 [00:08<27:33,  1.52s/it]











  1%|          | 7/1093 [00:09<23:39,  1.31s/it]











  1%|          | 8/1093 [00:11<24:02,  1.33s/it]











  1%|          | 9/1093 [00:12<21:35,  1.20s/it]











  1%|          | 10/1093 [00:13<20:39,  1.14s/it]











  1%|          | 11/1093 [00:14<19:35,  1.09s/it]











  1%|          | 12/1093 [00:14<18:37,  1.03s/it]











  1%|          | 13/1093 [00:16<19:54,  1.11s/it]











  1%|▏         | 14/1093 [00:17<20:10,  1.12s/it]











  1%|▏         | 15/1093 [00:18<20:52,  1.16s/it]











  1%|▏         | 16/1093 [00:1

  8%|▊         | 84/1093 [01:27<17:05,  1.02s/it]











  8%|▊         | 85/1093 [01:28<16:30,  1.02it/s]











  8%|▊         | 86/1093 [01:28<15:25,  1.09it/s]











  8%|▊         | 87/1093 [01:29<14:24,  1.16it/s]











  8%|▊         | 88/1093 [01:31<17:54,  1.07s/it]











  8%|▊         | 89/1093 [01:31<13:59,  1.20it/s]











  8%|▊         | 90/1093 [01:32<14:28,  1.15it/s]











  8%|▊         | 91/1093 [01:33<17:20,  1.04s/it]











  8%|▊         | 92/1093 [01:35<17:56,  1.08s/it]











  9%|▊         | 93/1093 [01:36<19:23,  1.16s/it]











  9%|▊         | 94/1093 [01:37<19:41,  1.18s/it]











  9%|▊         | 95/1093 [01:38<17:35,  1.06s/it]











  9%|▉         | 96/1093 [01:39<16:40,  1.00s/it]











  9%|▉         | 97/1093 [01:40<16:20,  1.02it/s]











  9%|▉         | 98/1093 [01:41<16:29,  1.01it/s]











  9%|▉         | 99/1093 [01:42<17:38,  1.06s/it]











  9%|▉         | 100/109

 15%|█▌        | 167/1093 [02:50<18:44,  1.21s/it]











 15%|█▌        | 168/1093 [02:52<22:33,  1.46s/it]











 15%|█▌        | 169/1093 [02:53<22:13,  1.44s/it]











 16%|█▌        | 170/1093 [02:54<19:08,  1.24s/it]











 16%|█▌        | 171/1093 [02:54<15:35,  1.01s/it]











 16%|█▌        | 172/1093 [02:55<13:36,  1.13it/s]

1084 crashed














 16%|█▌        | 173/1093 [04:30<7:25:24, 29.05s/it]











 16%|█▌        | 174/1093 [04:31<5:15:55, 20.63s/it]











 16%|█▌        | 175/1093 [04:32<3:45:27, 14.74s/it]











 16%|█▌        | 176/1093 [04:32<2:40:21, 10.49s/it]











 16%|█▌        | 177/1093 [04:33<1:56:00,  7.60s/it]











 16%|█▋        | 178/1093 [04:34<1:23:41,  5.49s/it]











 16%|█▋        | 179/1093 [04:35<1:03:17,  4.15s/it]











 16%|█▋        | 180/1093 [04:35<48:26,  3.18s/it]  











 17%|█▋        | 181/1093 [04:37<41:16,  2.72s/it]











 17%|█▋        | 182/1093 [04:39<37:18,  2.46s/it]











 17%|█▋        | 183/1093 [04:40<32:23,  2.14s/it]











 17%|█▋        | 184/1093 [04:41<26:38,  1.76s/it]











 17%|█▋        | 185/1093 [04:42<23:03,  1.52s/it]











 17%|█▋        | 186/1093 [04:43<20:53,  1.38s/it]











 17%|█▋        | 187/1093 [04:44<20:03,  1.33s/it]











 17%|█▋        | 188/1093 [04:45<18:29,  1

...

1741 crashed














 76%|███████▌  | 830/1093 [20:09<58:04, 13.25s/it]











 76%|███████▌  | 831/1093 [20:12<44:04, 10.09s/it]











 76%|███████▌  | 832/1093 [20:13<32:47,  7.54s/it]











 76%|███████▌  | 833/1093 [20:14<23:53,  5.52s/it]











 76%|███████▋  | 834/1093 [20:15<17:39,  4.09s/it]











 76%|███████▋  | 835/1093 [20:16<13:44,  3.19s/it]











 76%|███████▋  | 836/1093 [20:17<10:58,  2.56s/it]











 77%|███████▋  | 837/1093 [20:18<08:51,  2.08s/it]











 77%|███████▋  | 838/1093 [20:19<07:11,  1.69s/it]











 77%|███████▋  | 839/1093 [20:20<06:35,  1.56s/it]











 77%|███████▋  | 840/1093 [20:21<05:36,  1.33s/it]











 77%|███████▋  | 841/1093 [20:21<04:27,  1.06s/it]











 77%|███████▋  | 842/1093 [20:23<05:06,  1.22s/it]











 77%|███████▋  | 843/1093 [20:23<04:23,  1.05s/it]











 77%|███████▋  | 844/1093 [20:25<04:33,  1.10s/it]











 77%|███████▋  | 845/1093 [20:26<04:51,  1.17s/it]









 84%|████████▎ | 913/1093 [21:43<03:30,  1.17s/it]











 84%|████████▎ | 914/1093 [21:44<03:29,  1.17s/it]











 84%|████████▎ | 915/1093 [21:45<03:29,  1.18s/it]











 84%|████████▍ | 916/1093 [21:46<03:32,  1.20s/it]











 84%|████████▍ | 917/1093 [21:47<03:27,  1.18s/it]











 84%|████████▍ | 918/1093 [21:49<03:36,  1.24s/it]











 84%|████████▍ | 919/1093 [21:49<03:00,  1.04s/it]











 84%|████████▍ | 920/1093 [21:51<03:12,  1.11s/it]











 84%|████████▍ | 921/1093 [21:51<02:57,  1.03s/it]











 84%|████████▍ | 922/1093 [21:52<02:36,  1.09it/s]











 84%|████████▍ | 923/1093 [21:53<02:29,  1.14it/s]











 85%|████████▍ | 924/1093 [21:54<02:25,  1.16it/s]











 85%|████████▍ | 925/1093 [21:55<02:32,  1.10it/s]











 85%|████████▍ | 926/1093 [21:56<02:32,  1.10it/s]











 85%|████████▍ | 927/1093 [21:56<02:22,  1.16it/s]











 85%|████████▍ | 928/1093 [21:57<02:12,  1.24it/s]











 85%|███

 91%|█████████ | 996/1093 [23:06<01:01,  1.57it/s]











 91%|█████████ | 997/1093 [23:07<01:11,  1.35it/s]











 91%|█████████▏| 998/1093 [23:07<01:10,  1.36it/s]











 91%|█████████▏| 999/1093 [23:08<01:19,  1.19it/s]











 91%|█████████▏| 1000/1093 [23:09<01:14,  1.25it/s]











 92%|█████████▏| 1001/1093 [23:10<01:23,  1.10it/s]











 92%|█████████▏| 1002/1093 [23:11<01:22,  1.10it/s]











 92%|█████████▏| 1003/1093 [23:12<01:20,  1.12it/s]











 92%|█████████▏| 1004/1093 [23:13<01:14,  1.19it/s]











 92%|█████████▏| 1005/1093 [23:14<01:36,  1.10s/it]











 92%|█████████▏| 1006/1093 [23:15<01:33,  1.07s/it]











 92%|█████████▏| 1007/1093 [23:17<01:36,  1.13s/it]











 92%|█████████▏| 1008/1093 [23:18<01:34,  1.11s/it]











 92%|█████████▏| 1009/1093 [23:19<01:36,  1.14s/it]











 92%|█████████▏| 1010/1093 [23:20<01:34,  1.14s/it]











 92%|█████████▏| 1011/1093 [23:21<01:34,  1.15s/it]









 99%|█████████▊| 1078/1093 [24:28<00:14,  1.06it/s]











 99%|█████████▊| 1079/1093 [24:30<00:17,  1.24s/it]











 99%|█████████▉| 1080/1093 [24:31<00:13,  1.07s/it]











 99%|█████████▉| 1081/1093 [24:32<00:14,  1.18s/it]











 99%|█████████▉| 1082/1093 [24:33<00:12,  1.15s/it]











 99%|█████████▉| 1083/1093 [24:35<00:14,  1.46s/it]











 99%|█████████▉| 1084/1093 [24:36<00:11,  1.23s/it]











 99%|█████████▉| 1085/1093 [24:37<00:09,  1.20s/it]











 99%|█████████▉| 1086/1093 [24:38<00:07,  1.09s/it]











 99%|█████████▉| 1087/1093 [24:40<00:07,  1.20s/it]











100%|█████████▉| 1088/1093 [24:41<00:06,  1.27s/it]











100%|█████████▉| 1089/1093 [24:43<00:06,  1.62s/it]

In [ ]:
SELECT linguist.add_locales_ngrams('[
   {
       "word_id":10003175,
       "ngramm":"неопределенно угрожающий",
       "ref_id":1,
       "setting_id":1,
       "simlar_words":[
           {
               "word_id":"17720",
               "ref_id":"1",
               "setting_id":"1",
               "ngramm":"искренне неподдельно"
           },
           {
               "word_id":"21761",
               "ref_id":"1",
               "setting_id":"2",
               "ngramm":"учреждение заведение"
           }
      ]
   },
   {
       "word_id":10003491,
       "ngramm":"деревянные клинья",
       "ref_id":1,
       "setting_id":1,
       "simlar_words":[
           {
               "word_id":"24813",
               "ref_id":"1",
               "setting_id":"1",
               "ngramm":"деревянный молоток"
           },
           {
               "word_id":"53843",
               "ref_id":"1",
               "setting_id":"4",
               "ngramm":"ужасный страшный" }
        ]
}
]'::jsonb)

# CSV APPROACH

In [42]:
def insert(directory):
    conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
    cursor = conn.cursor()
    files_list = os.listdir(directory)
    sim_words_list = []
    for unigr_index in tqdm(range(len(files_list))):
        unigr = files_list[unigr_index]
        if unigr.endswith("json"):
            with open(os.path.join(directory,unigr)) as f:
                main_words = json.load(f)
                
                for main_word in main_words:
                    main_word_id = main_word['word_id']
                    main_ref = main_word['ref_id']
                    main_set = main_word['setting_id']
                    orig_word = main_word['word']
                    
                    for sim_word in main_word['simlar_words']:
                        sim_words_list.append((main_word_id, main_ref, main_set, 0, 0,0,sim_word,orig_word))
                        #print(main_word_id, main_ref, main_set, sim_word)
                        #cursor.execute("INSERT INTO linguist.mix_locales (word_id, ref_id, setting_id, mix_word_id, mix_ref_id, mix_setting_id, mix_word,orig_word) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", (main_word_id, main_ref, main_set, 0, 0,0,sim_word,orig_word))
                        #conn.commit() 
                        #break
            #print(unigr,"all words collected, going to insert")
            #cursor.executemany("INSERT INTO linguist.mix_locales (word_id, ref_id, setting_id, mix_word_id, mix_ref_id, mix_setting_id, mix_word,orig_word) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", sim_words_list)
            #conn.commit()
            #print("inseted completed")
                    #break
        #break
    #cursor.close()
    #conn.close()
    return sim_words_list
        
words = insert("./save_unigr_rus_2")






  0%|          | 0/3187 [00:00<?, ?it/s]




  1%|          | 16/3187 [00:00<00:19, 158.97it/s]




  1%|▏         | 44/3187 [00:00<00:17, 181.85it/s]




  2%|▏         | 67/3187 [00:00<00:16, 193.53it/s]




  3%|▎         | 92/3187 [00:00<00:15, 206.20it/s]




  4%|▍         | 124/3187 [00:00<00:13, 230.37it/s]




  5%|▍         | 155/3187 [00:00<00:12, 247.87it/s]




  6%|▌         | 186/3187 [00:00<00:11, 262.97it/s]




  7%|▋         | 213/3187 [00:00<00:14, 207.88it/s]




  8%|▊         | 240/3187 [00:01<00:13, 222.93it/s]




  8%|▊         | 269/3187 [00:01<00:12, 239.42it/s]




  9%|▉         | 300/3187 [00:01<00:11, 255.99it/s]




 10%|█         | 329/3187 [00:01<00:10, 264.57it/s]




 11%|█         | 357/3187 [00:01<00:10, 267.07it/s]




 12%|█▏        | 385/3187 [00:01<00:10, 261.84it/s]




 13%|█▎        | 412/3187 [00:01<00:10, 256.83it/s]




 14%|█▍        | 443/3187 [00:01<00:10, 270.50it/s]




 15%|█▍        | 476/3187 [00:01<00:09, 285.37it/s]




 1

In [68]:
words_first = insert("./save_unigr_rus")






  0%|          | 0/2005 [00:00<?, ?it/s]




  0%|          | 2/2005 [00:00<01:52, 17.82it/s]




  1%|          | 20/2005 [00:00<01:21, 24.39it/s]




  2%|▏         | 38/2005 [00:00<00:59, 32.88it/s]




  3%|▎         | 56/2005 [00:00<00:44, 43.48it/s]




  4%|▎         | 75/2005 [00:00<00:34, 56.54it/s]




  4%|▍         | 89/2005 [00:02<01:32, 20.65it/s]




  5%|▌         | 109/2005 [00:02<01:07, 28.23it/s]




  6%|▋         | 130/2005 [00:02<00:49, 38.05it/s]




  8%|▊         | 151/2005 [00:02<00:36, 50.42it/s]




  8%|▊         | 168/2005 [00:02<00:30, 60.78it/s]




  9%|▉         | 184/2005 [00:02<00:25, 71.99it/s]




 10%|█         | 201/2005 [00:02<00:20, 86.74it/s]




 11%|█         | 221/2005 [00:02<00:17, 104.41it/s]




 12%|█▏        | 243/2005 [00:03<00:14, 123.12it/s]




 13%|█▎        | 268/2005 [00:03<00:12, 144.71it/s]




 14%|█▍        | 288/2005 [00:03<00:10, 156.62it/s]




 15%|█▌        | 308/2005 [00:03<00:10, 155.07it/s]




 16%|█▋        | 

In [69]:
df_orig = pd.DataFrame(words_first, columns =['word_id', 'ref_id', 'setting_id', 
                                   'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 


In [70]:
df_orig.to_csv("unigr_2.csv", index = False)

In [72]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7230569 entries, 0 to 7230568
Data columns (total 8 columns):
word_id           int64
ref_id            int64
setting_id        int64
mix_word_id       int64
mix_ref_id        int64
mix_setting_id    int64
mix_word          object
orig_word         object
dtypes: int64(6), object(2)
memory usage: 441.3+ MB


In [82]:
for i in range(0, len(df_orig), 500000):
    print(i,i+500000)
    df = pd.DataFrame(words_first[i:i+500000], columns =['word_id', 'ref_id', 'setting_id', 
                                   'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 
    df.to_csv("./unigr2(actual1)/unigr1_" + str(i) + "-" + str(i+500000) + ".csv", index = False)


0 500000
500000 1000000
1000000 1500000
1500000 2000000
2000000 2500000
2500000 3000000
3000000 3500000
3500000 4000000
4000000 4500000
4500000 5000000
5000000 5500000
5500000 6000000
6000000 6500000
6500000 7000000
7000000 7500000


In [43]:
words[0]

(334870, 1, 1, 0, 0, 0, 'заглушать', 'разрушать')

In [64]:
df_big = pd.DataFrame(words, columns =['word_id', 'ref_id', 'setting_id', 'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 


In [65]:
df_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5827659 entries, 0 to 5827658
Data columns (total 8 columns):
word_id           int64
ref_id            int64
setting_id        int64
mix_word_id       int64
mix_ref_id        int64
mix_setting_id    int64
mix_word          object
orig_word         object
dtypes: int64(6), object(2)
memory usage: 355.7+ MB


In [80]:
for i in range(1500000, len(df_big), 500000):
    print(i,i+500000)
    df = pd.DataFrame(words[i:i+500000], columns =['word_id', 'ref_id', 'setting_id', 
                                   'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 
    df.to_csv("./unigr1(actual_2)/unigr1_" + str(i) + "-" + str(i+500000) + ".csv", index = False)


1500000 2000000
2000000 2500000
2500000 3000000
3000000 3500000
3500000 4000000
4000000 4500000
4500000 5000000
5000000 5500000
5500000 6000000


In [76]:
df = pd.DataFrame(words[1000000:1500000], columns =['word_id', 'ref_id', 'setting_id', 
                                   'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 


In [77]:
df.to_csv("unigr1_1000-1500.csv", index = False)

In [52]:
df.head()

,word_id,ref_id,setting_id,mix_word_id,mix_ref_id,mix_setting_id,mix_word,orig_word
0,334870,1,1,0,0,0,заглушать,разрушать
1,334870,1,1,0,0,0,прекращать,разрушать
2,334870,1,1,0,0,0,разбивать,разрушать
3,334870,1,1,0,0,0,управлять,разрушать
4,334870,1,1,0,0,0,раздевать,разрушать
